In [ ]:
#| hide
#| default_exp core
from fastcore.basics import patch

# core

> The telescope control is provided through the `Telescope` class which provides state tracking and low level methods - forming a basic API layer. The higher level functions are implemented as separate functions construced with the Telescope class API.

In [ ]:
#| exporti
import logging
import requests
from requests import session
import configparser
from os.path import expanduser
import diskcache
from bs4 import BeautifulSoup
import json
import time
import os, tempfile, shutil
from os import path
from zipfile import ZipFile, BadZipFile
from io import StringIO, BytesIO
from tqdm.auto import tqdm

In [ ]:
#| exporti
def cleanup(s):
    return s.encode('ascii','ignore').decode('ascii','ignore')

In [ ]:
#| export
class Telescope:
    '''
    Main API class
    '''
    
    url='https://www.telescope.org/'
    cameratypes={
        'constellation':'1',
        'galaxy':       '2',
        'cluster':      '3',
        'planet':'5',
        'coast':'6',
        'pirate':'7',
    }

    REQUESTSTATUS_TEXTS={
        1: "New",
        2: "New, allocated",
        3: "Waiting",
        4: "In progress",
        5: "Reallocate",
        6: "Waiting again",
        7: "Complete on site",
        8: "Complete",
        9: "Hold",
        10: "Frozen",
        20: "Expired",
        21: "Expired w/CJobs",
        22: "Cancelled",
        23: "Cancelled w/CJobs",
        24: "Invalid",
        25: "Never rises",
        26: "Other error",
    }
    
    def __init__(self, user, passwd, cache='.cache/jobs'):
        self.s=None
        self.user=user
        self.passwd=passwd
        self.tout=60
        self.retry=15
        self.login()
        self.cache=cache


In [ ]:
#| exporti
@patch
def login(self: Telescope):
    log = logging.getLogger(__name__)
    payload = {'action': 'login',
               'username': self.user,
               'password': self.passwd,
               'stayloggedin': 'true'}
    log.debug('Get session ...')
    self.s=session()
    log.debug('Logging in ...')
    self.s.post(self.url+'login.php', data=payload)

In [ ]:
#| exporti
@patch
def logout(self: Telescope):
    if self.s is None :
        self.s.post(self.url+'logout.php')
        self.s=None

In [ ]:
#| login
config = configparser.ConfigParser()
config.read(expanduser('~/.config/telescope.ini'))

scope=Telescope(config['telescope.org']['user'],
                    config['telescope.org']['password'],
                    config['cache']['jobs'])
astrometryAPIkey=config['astrometry.net']['apikey']

wcscache=diskcache.Cache(config['cache']['wcs'])
seqcache=diskcache.Cache(config['cache']['seq'])

In [ ]:
#| export
@patch
def get_user_requests(self: Telescope, sort='rid', folder=1):
    '''
    Get all user requests from folder (Inbox=1 by default),
    sorted by sort column ('rid' by default). 
    Possible sort columns are: 'rid', 'object', 'completion'
    The data is returned as a list of dictionaries.
    '''

    #fetch first batch        
    params={
        'limit': 100,
        'sort': sort,
        'folderid': folder}

    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})
    res=[]
    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']

    # Fetch the rest
    params['limit']=total-len(res)
    params['startAfterRow']=len(res)
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "1-get-list-own",
                                               'params' : json.dumps(params)})

    dat=json.loads(rq.content)
    total=int(dat['data']['totalRequests'])
    res+=dat['data']['requests']
    return res

In [ ]:
#| login
reqs = scope.get_user_requests()
print(f'User {scope.user} has {len(reqs)} requests. Recent few:')
for rq in reqs[:10]:
    print(f'{rq["id"]}: {rq["objectname"]}')

User jochym has 1505 requests. Recent few:
748337: DX Vul
748246: LX Cyg
748245: SS Cyg
747836: DQ Vul
747835: DX Vul
747819: EQ Lyr
747818: V686 Cyg
747817: IP Cyg
747816: SS Cyg
747815: CH Cyg


In [ ]:
#| export
@patch
def get_user_folders(self: Telescope):
    '''
    Get all user folders. Returns list of dictionaries.
    '''
    rq = self.s.post(self.url+"api-user.php", {'module': "request-manager", 
                                               'request': "0-get-my-folders"})
    return json.loads(rq.content)['data']

In [ ]:
#| login
scope.get_user_folders()

[{'id': '1', 'creationtime': '0', 'name': 'Inbox', 'count': '1505'},
 {'id': '2', 'creationtime': '0', 'name': 'Favourites', 'count': None},
 {'id': '3', 'creationtime': '0', 'name': 'Archive', 'count': '447'},
 {'id': '4', 'creationtime': '0', 'name': 'Trash', 'count': '52'},
 {'id': '461',
  'creationtime': '1407254495',
  'name': 'Complete',
  'count': '13'}]

In [ ]:
#| export
@patch
def get_obs_list(self: Telescope, t=None, dt=1, filtertype='', camera='', hour=16, minute=0):
    '''Get the dt days of observations taken no later then time in t.

        ### Input
        
        t  - end time in seconds from the epoch
            (as returned by time.time())
        dt - number of days, default to 1
        filtertype - filter by type of filter used
        camera - filter by the camera/telescope used

        ### Output
        
        Returns a list of JobIDs (int) for the observations.

    '''

    assert(self.s is not None)

    if t is None :
        t=time.time()-time.timezone


    st=time.gmtime(t-86400*dt)
    et=time.gmtime(t)

    d=st.tm_mday
    m=st.tm_mon
    y=st.tm_year
    de=et.tm_mday
    me=et.tm_mon
    ye=et.tm_year

    log = logging.getLogger(__name__)
    log.debug('%d/%d/%d -> %d/%d/%d', d,m,y,de,me,ye)

    try :
        telescope=self.cameratypes[camera.lower()]
    except KeyError:
        telescope=''

    searchdat = {
        'sort1':'completetime',
        'sort1order':'desc',
        'searchearliestcom[]':[d, m, y, str(hour),str(minute)],
        'searchlatestcom[]':  [de,me,ye,str(hour),str(minute)],
        'searchstatus[]':['1'],
        'resultsperpage':'1000',
        'searchfilter':filtertype,
        'searchtelescope':telescope,
        'submit':'Go'
    }

    headers = {'Content-Type': 'application/x-www-form-urlencoded'}


    request = self.s.post(self.url+'v3job-search-query.php',
                     data=searchdat, headers=headers)
    soup = BeautifulSoup(request.text,'lxml')

    jlst=[]
    for l in soup.findAll('tr'):
        try :
            a=l.find('a').get('href')
        except AttributeError :
            continue
        jid=a.rfind('jid')
        if jid>0 :
            jid=a[jid+4:].split('&')[0]
            jlst.append(int(jid))
    return jlst

In [ ]:
#| login 
jobs = scope.get_obs_list()
print(jobs)

[402894, 402772, 402591, 402560, 402933, 402773, 402586, 402563, 402947, 402835, 402848, 402734, 402733, 402732, 402541, 402656, 402797, 402799, 402718, 402730, 402647, 402599, 402340, 402362, 402536, 402376]


In [ ]:
#| export
@patch
def get_job(self: Telescope, jid=None):
    '''Get a job data for a given JID'''

    assert(jid is not None)
    assert(self.s is not None)

    log = logging.getLogger(__name__)
    log.debug(jid)

    obs={}
    obs['jid']=jid
    rq=self.s.post(self.url+('v3cjob-view.php?jid=%d' % jid))
    soup = BeautifulSoup(rq.text, 'lxml')
    for l in soup.findAll('tr'):
        log.debug(cleanup(l.text))
        txt=''
        for f in l.findAll('td'):
            if txt.find('Object Type') >= 0:
                obs['type']=f.text
            if txt.find('Object ID') >= 0:
                obs['oid']=f.text
            if txt.find('Telescope Type Name') >= 0:
                obs['tele']=f.text
            if txt.find('Filter Type') >= 0:
                obs['filter']=f.text
            if txt.find('Exposure Time') >= 0:
                obs['exp']=f.text
            if txt.find('Completion Time') >= 0:
                t=f.text.split()
                obs['completion']=t[3:6]+[t[6][1:]]+[t[7][:-1]]
            if txt.find('Status') >= 0:
                obs['status']= (f.text == 'Success')

            txt=f.text
    log.info('%(jid)d [%(tele)s, %(filter)s, %(status)s]: %(type)s %(oid)s %(exp)s', obs)

    return obs

In [ ]:
#| login
job = scope.get_job(jobs[1])
print(job)

{'jid': 402772, 'type': 'NGC', 'oid': '271', 'exp': '120000 ms', 'filter': 'Clear', 'tele': 'Galaxy', 'completion': ['5', 'December', '2022', '22:09:30', 'UTC'], 'status': True}


In [ ]:
#| export
@patch
def download_obs(self: Telescope, obs=None, directory='.', cube=True, pbar=False):
    '''Download the raw observation obs (obtained from get_job) into 3D fits
    file named jid.fits located in the directory (current by default).
    Alternatively, when the cube=False the file will be a zip of 3 fits files.
    The name of the file (without directory) is returned.
    
    The zip API got dropped from telescope.org and it stoped working. 
    
    '''

    assert(obs is not None)
    assert(self.s is not None)

    if not cube:
        print('The zip output is no longer supported!')
        return None
    
    chunksize = 1024
    rq=self.s.get(self.url+
                  ('v3image-download%s.php?jid=%d' %
                    ('' if cube else '-layers', obs['jid'])),
                  stream=True)
    
    size = int(rq.headers.get('Content-Length', 0))
    tq = None
    fn = ('%(jid)d.' % obs) + ('fits' if cube else 'zip')
    if pbar :
        tq = tqdm(desc=fn,       
                  total=size,       
                  unit="B",       
                  unit_scale=True,        
                  leave=True,       
                  miniters=1)
    
    with open(path.join(directory, fn), 'wb') as fd:
        for chunk in rq.iter_content(chunksize):
            if chunk:
                fd.write(chunk)
                if tq :
                    tq.update(len(chunk))
    if tq :
        tq.close()
    return fn


In [ ]:
#| login
scope.download_obs(job, directory='OUT', pbar=True)

402772.fits:   0%|          | 0.00/4.73M [00:00<?, ?B/s]

'402772.fits'

In [ ]:
#| export
@patch
def get_obs(self: Telescope, obs=None, cube=True, recurse=True, pbar=False):
    '''Get the raw observation obs (obtained from get_job) into zip
    file-like object. The function returns ZipFile structure of the
    downloaded data.'''

    assert(obs is not None)
    assert(self.s is not None)

    log = logging.getLogger(__name__)

    fn = ('%(jid)d.' % obs) + ('fits' if cube else 'zip')
    fp = path.join(self.cache,fn[0],fn[1],fn)
    if not path.isfile(fp) :
        log.info('Getting %s from server', fp)
        os.makedirs(path.dirname(fp), exist_ok=True)
        self.download_obs(obs,path.dirname(fp),cube=cube,pbar=pbar)
    else :
        log.info('Getting %s from cache', fp)
    content = open(fp,'rb')
    try :
        return content if cube else ZipFile(content)
    except BadZipFile :
        # Probably corrupted download. Try again once.
        content.close()
        os.remove(fp)
        if recurse :
            return self.get_obs(obs, cube, False)
        else :
            return None


In [ ]:
#| login
scope.get_obs(job, pbar=True)

<_io.BufferedReader name='.cache/jobs/4/0/402772.fits'>

In [ ]:
#| export
@patch
def download_obs_processed(self: Telescope, obs=None, directory='.', cube=False, pbar=False):
    '''Download the raw observation obs (obtained from get_job) into zip
    file named job_jid.zip located in the directory (current by default).
    Alternatively, when the cube=True the file will be a 3D fits file.
    The name of the file (without directory) is returned.'''

    assert(obs is not None)
    assert(self.s is not None)

    log = logging.getLogger(__name__)

    fn=None

    tout=self.tout
    tq = None
    chunksize = 1024
 
    while tout > 0 :
        rq=self.s.get(self.url+
                      ('imageengine-request.php?jid=%d&type=%d' %
                        (obs['jid'], 1 if cube else 3 )))

        soup = BeautifulSoup(rq.text, 'lxml')
        dlif=soup.find('iframe')

        try :
            dl=dlif.get('src')
            rq=self.s.get(self.url+dl,stream=True)
            size = int(rq.headers.get('Content-Length', 0))
            fn = ('art_%(jid)d.' % obs) + ('fits' if cube else 'zip')

            if pbar :
                tq = tqdm(desc=fn,       
                          total=size,       
                          unit="B",       
                          unit_scale=True,        
                          leave=True,       
                          miniters=1)

            with open(path.join(directory, fn), 'wb') as fd:
                for chunk in rq.iter_content(chunksize):
                    fd.write(chunk)
                    if tq :
                        tq.update(len(chunk))
                    
            if tq: 
                tq.close()
            return fn
        except AttributeError :
            tout-=self.retry
            log.warning('No data. Sleep for %ds ...'%self.retry)
            time.sleep(self.retry)

    return None



In [ ]:
#| login
print(scope.download_obs_processed(job, directory='OUT', cube=True, pbar=True))
print(scope.download_obs_processed(job, directory='OUT', cube=False, pbar=True))

art_402772.fits:   0%|          | 0.00/4.73M [00:00<?, ?B/s]

art_402772.fits


art_402772.zip:   0%|          | 0.00/3.53M [00:00<?, ?B/s]

art_402772.zip


In [ ]:
#| export
@patch
def get_obs_processed(self: Telescope, obs=None, cube=False):
    '''Get the raw observation obs (obtained from get_job) into zip
    file-like object. The function returns ZipFile structure of the
    downloaded data.'''

    assert(obs is not None)
    assert(self.s is not None)
    log = logging.getLogger(__name__)

    tout=self.tout
       
    while tout > 0 :
        rq=self.s.get(self.url+
                      ('imageengine-request.php?jid=%d&type=%d' %
                        (obs['jid'], 1 if cube else 3 )))

        soup = BeautifulSoup(rq.text,'lxml')
        dlif=soup.find('iframe')
        try :
            dl=dlif.get('src')
            rq=self.s.get(self.url+dl,stream=True)
            return BytesIO(rq.content) if cube else ZipFile(BytesIO(rq.content))

        except AttributeError :
            tout-=self.retry
            log.warning('No data. Sleep for %ds ...'%self.retry)
            time.sleep(self.retry)

    return None


In [ ]:
#| login
scope.get_obs_processed(job, cube=True)

<_io.BytesIO>

In [ ]:
#| login
scope.get_obs_processed(job, cube=False)

<zipfile.ZipFile file=<_io.BytesIO object> mode='r'>